Modelado Predictivo de Precios de Autos
Autor: Hector Vejar Casillas

Carga de datos

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

In [2]:
#https://www.kaggle.com/datasets/asinow/car-price-dataset
carDataset = pd.read_csv('./car_price_dataset.csv')
carDataset.head()
#No es necesario verificar  los valores nulos y demas ya que en el anterior analisis se valido todo eso.

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867


Separacion de los datos para entrenamiento y validacion.

In [4]:
y=carDataset['Price']
X= carDataset.drop('Price',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

Manejo de las columnas categoricas | Encoding 

In [9]:
#Para obtener los nombres de las columnas con datos categoricos
s = (X_train.dtypes == 'object')
object_cols1 = list(s[s].index)
s = (X_train.dtypes == 'numeric')
object_cols2 = list(s[s].index)
# Crear un transformador para las variables categóricas y numericas
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), object_cols1), ('num', StandardScaler(), object_cols2)   ],
    remainder='passthrough'  # Deja las columnas numéricas sin cambios
)

Declaramos el pipeline que contiene el modelo

In [31]:
pipe = make_pipeline( preprocessor ,DecisionTreeRegressor())

Calculamos el score de nuestro modelo

In [15]:
scores = -1 * cross_val_score(pipe, X, y, cv=5 , scoring='neg_mean_absolute_error')
print(scores.mean())

464.3607


In [33]:
scores =  cross_val_score(pipe, X, y, cv=5 , scoring='r2')
print(f"Porcentaje de efectividad {scores.mean() * 100}")

Porcentaje de efectividad 96.15513465063145


En promedio las predicciones del modelo tienen un error de 464.3607 en cuanto ala prediccion del precio de los automobiles 

Ahora intentare con otro modelo XGboost
pip install xgboost


In [21]:
pipe = make_pipeline( preprocessor ,xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
scores = -1 * cross_val_score(pipe, X, y, cv=5 , scoring='neg_mean_absolute_error')
print(scores.mean())

186.13496920166014


In [35]:
scores =  cross_val_score(pipe, X, y, cv=5 , scoring='r2')
print(f"Porcentaje de efectividad {scores.mean() * 100}")

Porcentaje de efectividad 96.16516020142821
